# Text to SQL using T5

# Import and install necessary packages

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from evaluate import load
import numpy as np

from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


# Load the dataset

In [ ]:
# Load the dataset
dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/Data/text_to_sql_data/train.csv',
    'validation': '/content/drive/MyDrive/Data/text_to_sql_data/validation.csv',
    'test': '/content/drive/MyDrive/Data/text_to_sql_data/test.csv'
    })

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Load the model and tokenizer

In [ ]:
# Initialize tokenizer and model
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Preprocess data

In [ ]:
# Preprocessing function
def preprocess_function(examples):
    inputs = ["translate English to SQL: " + p + " " + c for p, c in zip(examples['prompt'], examples['context'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/133920 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/26784 [00:00<?, ? examples/s]

Map:   0%|          | 0/17856 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.025500,0.020345,0.267506
2,0.022300,0.017092,0.270813
3,0.020600,0.016268,0.271898


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=50220, training_loss=0.032301129690920014, metrics={'train_runtime': 16774.2869, 'train_samples_per_second': 23.951, 'train_steps_per_second': 2.994, 'total_flos': 2.446549221113856e+17, 'train_loss': 0.032301129690920014, 'epoch': 3.0})

# Save the model

In [ ]:
# Save the model
trainer.save_model("t5_sql_model")
torch.save(trainer.optimizer.state_dict(), "t5_sql_model/optimizer.pt")
!cp -r t5_sql_model "/content/drive/My Drive/"

# Generate examples

In [ ]:
# Function to generate SQL queries
def generate_sql(prompt, schema):
    input_text = "translate English to SQL: " + prompt + " " + schema
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    # Move inputs to the GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}
    max_output_length = 1024
    outputs = model.generate(**inputs, max_length=max_output_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Interactive loop
print("Enter 'quit' to exit.")
while True:
    prompt = input("Insert prompt: ")
    schema = input("Insert schema: ")
    if prompt.lower() == 'quit':
        break

    sql_query = generate_sql(prompt, schema)
    print(f"Generated SQL query: {sql_query}")
    print()

In [ ]:
# Print examples
num_examples = 20
print(f"\nShowing {num_examples} example predictions:")

for i in range(min(num_examples, len(dataset["test"]))):
    prompt = dataset["test"][i]['prompt']
    schema = dataset["test"][i]['context']
    actual_sql = dataset["test"][i]['answer']
    predicted_sql = generate_sql(prompt, schema)

    print(f"\nExample {i+1}:")
    print(f"Prompt: {prompt}")
    print(f"Schema: {schema}")
    print(f"Actual SQL: {actual_sql}")
    print(f"Predicted SQL: {predicted_sql}")


Showing 20 example predictions:

Example 1:
Prompt: What is the name of the school with the Tornadoes Lady Tornadoes?
Schema: CREATE TABLE table_name_41 (school VARCHAR, nickname_s_ VARCHAR);
Actual SQL: SELECT school FROM table_name_41 WHERE nickname_s_ = "tornadoes lady tornadoes";
Predicted SQL: SELECT school FROM table_name_41 WHERE nickname_s_ = "tornadoes lady tornadoes";

Example 2:
Prompt: What is the total number of hours spent on open pedagogy projects per student?
Schema: CREATE TABLE students (student_id INT, student_name TEXT);  CREATE TABLE open_pedagogy_projects (project_id INT, student_id INT, hours_spent_on_project INT); 
Actual SQL: SELECT s.student_name, SUM(opp.hours_spent_on_project) as total_hours_spent_on_projects FROM open_pedagogy_projects opp JOIN students s ON opp.student_id = s.student_id GROUP BY opp.student_id;
Predicted SQL: SELECT s.student_name, SUM(op.hours_spent_on_project) FROM students s JOIN open_pedagogy_projects op ON s.student_id = op.student_i